In [3]:
try:
    from pyspark.sql import SparkSession
except:
    import findspark
    findspark.init()
    from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("row_number") \
       .getOrCreate() 

In [4]:
from pyspark.sql import functions as F

In [19]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
import pyspark.sql.functions as F


spark = SparkSession.builder.getOrCreate()

df = spark.createDataFrame([(1, "Maria","2023-05-07 00:52:11","2023-07-01","1"), 
                            (1, "Maria","2023-07-07 10:52:11","2023-07-01","1"), 
                            (1, "Maria","2023-07-08 10:52:11","2023-07-01","2"),
    (3, "Charlie","2023-07-01 00:52:11","2023-07-02","9000"), (3, "David","2023-07-01 00:53:11","2023-07-02","5000")],
                           ["id", "name","hora","date","atraso"])

windowSpec = Window.partitionBy().orderBy("id")

df_with_row_number = df.withColumn("orden_columnas", F.row_number().over(windowSpec))

# Show the result
df_with_row_number.show()

+---+-------+-------------------+----------+-------+--------------+
| id|   name|               hora|      date|salario|orden_columnas|
+---+-------+-------------------+----------+-------+--------------+
|  1|  Alice|2023-07-01 00:52:11|2023-07-01|   8000|             1|
|  1|  Alice|2023-07-01 10:52:11|2023-07-01|   9000|             2|
|  3|Charlie|2023-07-01 00:52:11|2023-07-02|   9000|             3|
|  3|  David|2023-07-01 00:53:11|2023-07-02|   5000|             4|
+---+-------+-------------------+----------+-------+--------------+



In [ ]:
["jose","20000","malo","end_date":2024-10-23]--->clausura
["jose","30000","bueno","end_date":""]--->vigente

SCD-1
SCD-2

In [20]:
df=df.withColumn("load_timestamp",F.to_timestamp(F.col("hora"),"yyyy-MM-dd HH:mm:ss"))\
    .withColumn("fecha",F.to_date(F.col("date"),"yyyy-MM-dd"))
df.show(5,truncate=False)

+---+-------+-------------------+----------+-------+-------------------+----------+
|id |name   |hora               |date      |salario|load_timestamp     |fecha     |
+---+-------+-------------------+----------+-------+-------------------+----------+
|1  |Alice  |2023-07-01 00:52:11|2023-07-01|8000   |2023-07-01 00:52:11|2023-07-01|
|1  |Alice  |2023-07-01 10:52:11|2023-07-01|9000   |2023-07-01 10:52:11|2023-07-01|
|3  |Charlie|2023-07-01 00:52:11|2023-07-02|9000   |2023-07-01 00:52:11|2023-07-02|
|3  |David  |2023-07-01 00:53:11|2023-07-02|5000   |2023-07-01 00:53:11|2023-07-02|
+---+-------+-------------------+----------+-------+-------------------+----------+



In [21]:
windowSpec = Window.partitionBy("id").orderBy("load_timestamp")

df_with_row_number = df.withColumn("row_number", F.row_number().over(windowSpec))

df_with_row_number.show()

+---+-------+-------------------+----------+-------+-------------------+----------+----------+
| id|   name|               hora|      date|salario|     load_timestamp|     fecha|row_number|
+---+-------+-------------------+----------+-------+-------------------+----------+----------+
|  1|  Alice|2023-07-01 00:52:11|2023-07-01|   8000|2023-07-01 00:52:11|2023-07-01|         1|
|  1|  Alice|2023-07-01 10:52:11|2023-07-01|   9000|2023-07-01 10:52:11|2023-07-01|         2|
|  3|Charlie|2023-07-01 00:52:11|2023-07-02|   9000|2023-07-01 00:52:11|2023-07-02|         1|
|  3|  David|2023-07-01 00:53:11|2023-07-02|   5000|2023-07-01 00:53:11|2023-07-02|         2|
+---+-------+-------------------+----------+-------+-------------------+----------+----------+



In [12]:
df_with_row_number.orderBy(F.col("row_number").desc()).dropDuplicates((['id'])).show()

+---+-----+-------------------+----------+-------------------+----------+----------+
| id| name|               hora|      date|     load_timestamp|     fecha|row_number|
+---+-----+-------------------+----------+-------------------+----------+----------+
|  1|  Bob|2023-07-01 00:52:11|2023-07-01|2023-07-01 00:52:11|2023-07-01|         2|
|  3|David|2023-07-01 00:53:11|2023-07-02|2023-07-01 00:53:11|2023-07-02|         2|
+---+-----+-------------------+----------+-------------------+----------+----------+



In [55]:
df.selectExpr("ROW_NUMBER() OVER (ORDER BY load_timestamp NULLS LAST) - 1 AS orden", "id","name").show()

+-----+---+-------+
|orden| id|   name|
+-----+---+-------+
|    0|  1|  Alice|
|    1|  2|    Bob|
|    2|  3|Charlie|
|    3|  4|  David|
+-----+---+-------+



$$
f(\beta')>=f(\beta)+<z,\beta'-\beta>\\
f(\beta')>=0+<z,\beta'>\\
|\beta'|>=z\beta'\\
1) \beta'>=z\beta'\\
2) \beta'>=-z\beta'
$$